<a href="https://colab.research.google.com/github/ksnugroho/teori-praktik-cv-part1/blob/main/Bagian_1_Bab_2_2_8_Menyimpan_dan_Memuat_Model_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [3]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    def __len__(self):
        return len(self.x)

In [4]:
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [5]:
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

In [6]:
!pip -q install torch_summary
from torchsummary import summary

In [7]:
summary(model, torch.zeros(1,2))

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [8]:
import time

loss_func = nn.MSELoss()
opt = SGD(model.parameters(), lr=0.001)

loss_history = []

start = time.time()
for _ in range(50):
    for x, y in dl:
        opt.zero_grad()
        loss_value = loss_func(model(x), y)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value.item())
end = time.time()

print("Training time:", end - start)

Training time: 0.5430099964141846


# Menyimpan Model

In [9]:
save_path = 'mymodel.pth'
torch.save(model.state_dict(), save_path)

!du -hsc {save_path} # size of the model on disk

4.0K	mymodel.pth
4.0K	total


# Memuat Model

In [10]:
load_path = 'mymodel.pth'
model.load_state_dict(torch.load(load_path))

<ipython-input-10-2cb48ce5c835>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(load_path))


<All keys matched successfully>

# Prediksi

In [11]:
val = [[8,9],[10,11],[1.5,2.5]]
val = torch.tensor(val).float()

In [12]:
model(val.to(device))

tensor([[1.8434],
        [1.8434],
        [1.8154]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
val.sum(-1)

tensor([17., 21.,  4.])